In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re
import demoji
import emoji

In [2]:
STOP_WORD_FILE = 'Cung_cap_HV/vietnamese-stopwords.txt'

In [3]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

### Đọc dữ liệu, xem xét tổng quan dữ liệu

In [4]:
df = pd.read_csv('data_final_test_12_4_2023.csv')

In [5]:
df.head(5)

,product_id,description
0,190,áo ba lỗ chiều dài tay áo khác phong cách thể ...
1,191,áo ba lỗ xuất xứ việt nam tên tổ chức chịu trá...
2,192,áo ba lỗ thương hiệu tyasuo chiều dài tay áo k...
3,193,áo ba lỗ chất liệu cotton kho hàng gửi từ hà n...
4,1915,áo ba lỗ chiều dài tay áo không tay phong cách...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10576 entries, 0 to 10575
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   10576 non-null  int64 
 1   description  10576 non-null  object
dtypes: int64(1), object(1)
memory usage: 165.4+ KB


In [7]:
print(df.loc[0, 'description'])

áo ba lỗ chiều dài tay áo khác phong cách thể thao cơ bản đường phố nhiệt đới tall fit có xuất xứ việt nam dáng kiểu áo ôm sát chất liệu cotton mẫu sọc tăm có gân nổi kho hàng gửi từ bà rịa vũng tàu best tank tops for men áo ba lỗ quốc dân cho nam bảng bảng này là chuẩn mặc kiểu body nha quý khách còn nếu khách nào không thích mặc ôm thì tăng lên nhé còn muốn mặc rộng rộng thì tăng lên bảng chỉ mang tính chất tham khảo vì có nhiều khách thể hình không được cân đối để được tư vấn chính xác vui lòng inbox cho shop nhé kích thước vòng nách dài l dài xl dài dài bảng trên là mặc vừa cơ thể không quá ôm sát cũng không bị rộng quá vừa vừa người tuỳ vào sở thích mỗi người nếu bạn có body chuẩn múi muốn mặc ôm sát thì hạ xuống nếu muốn mặc rộng rãi thoải mái thì tăng thêm nhé bảng chỉ mang tính chất tham khảo nó chỉ tương đối chứ không chính xác tuyệt đối nha khách vì phụ thuộc vào cơ địa mỗi người khác nhau mọi trường hợp thắc mắc về sản phẩm quý khách vui lòng nhắn tin cho shop để được tư vấn

In [8]:
# word_tokenize
df["product_name_wt"] = df["description"].apply(lambda x: word_tokenize(x, format="text"))

In [9]:
df.head(2)

,product_id,description,product_name_wt
0,190,áo ba lỗ chiều dài tay áo khác phong cách thể ...,áo ba_lỗ chiều dài tay_áo khác phong_cách thể_...
1,191,áo ba lỗ xuất xứ việt nam tên tổ chức chịu trá...,áo ba lỗ xuất_xứ việt nam tên tổ_chức chịu trá...


In [10]:
# postag
products_postag = df["description"].apply(lambda x: pos_tag(x))

In [11]:
# sent_tokenize
products_sent = df["description"].apply(lambda x: sent_tokenize(x))

In [12]:
products_postag

0        [(áo, N), (ba lỗ, N), (chiều, N), (dài, A), (t...
1        [(áo, N), (ba, M), (lỗ, N), (xuất xứ, V), (việ...
2        [(áo, N), (ba, M), (lỗ, N), (thương hiệu, N), ...
3        [(áo, N), (ba, M), (lỗ, N), (chất liệu, V), (c...
4        [(áo, N), (ba lỗ, N), (chiều, N), (dài, A), (t...
                               ...                        
10571    [(vớ, N), (tất, V), (chất liệu, N), (cotton, V...
10572    [(vớ, N), (tất, V), (chất liệu, N), (cotton, V...
10573    [(vớ, N), (tất, V), (thương hiệu, N), (timan, ...
10574    [(vớ, N), (tất, V), (chất liệu, N), (cotton, V...
10575    [(vớ, N), (tất, V), (chất liệu, N), (cotton, V...
Name: description, Length: 10576, dtype: object

In [13]:
products_sent

0        [áo ba lỗ chiều dài tay áo khác phong cách thể...
1        [áo ba lỗ xuất xứ việt nam tên tổ chức chịu tr...
2        [áo ba lỗ thương hiệu tyasuo chiều dài tay áo ...
3        [áo ba lỗ chất liệu cotton kho hàng gửi từ hà ...
4        [áo ba lỗ chiều dài tay áo không tay phong các...
                               ...                        
10571    [vớ tất chất liệu cotton sợi dệt xuất xứ việt ...
10572    [vớ tất chất liệu cotton kiểu đóng gói đơn chi...
10573    [vớ tất thương hiệu timan chất liệu cotton xuấ...
10574    [vớ tất chất liệu cotton sợi tổng hợp xuất xứ ...
10575    [vớ tất chất liệu cotton xuất xứ việt nam kiểu...
Name: description, Length: 10576, dtype: object

In [14]:
df.to_csv('data_final_test_wt_12_4_2023.csv', index=False)

In [15]:
df_goc = pd.read_csv('Cung_cap_HV/Products_ThoiTrangNam_raw.csv')

In [16]:
df_full = pd.merge(df,df_goc,how='inner',on='product_id')

In [17]:
len(df_full)

10604

In [18]:
df_full = df_full.drop(['category','description_y'],axis=1)

In [19]:
df_full = df_full.rename(columns={'description_x': 'description'})

In [20]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10604 entries, 0 to 10603
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product_id       10604 non-null  int64  
 1   description      10604 non-null  object 
 2   product_name_wt  10604 non-null  object 
 3   product_name     10604 non-null  object 
 4   sub_category     10604 non-null  object 
 5   link             10604 non-null  object 
 6   image            10604 non-null  object 
 7   price            10604 non-null  float64
 8   rating           10604 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 828.4+ KB


In [21]:
df_full.to_csv('data_full_12_4_2023.csv', index = False)

# Gemsim

In [22]:
df_xl = pd.read_csv('data_final_test_wt_12_4_2023.csv')

In [23]:
df_xl.head()

,product_id,description,product_name_wt
0,190,áo ba lỗ chiều dài tay áo khác phong cách thể ...,áo ba_lỗ chiều dài tay_áo khác phong_cách thể_...
1,191,áo ba lỗ xuất xứ việt nam tên tổ chức chịu trá...,áo ba lỗ xuất_xứ việt nam tên tổ_chức chịu trá...
2,192,áo ba lỗ thương hiệu tyasuo chiều dài tay áo k...,áo ba lỗ thương_hiệu tyasuo chiều dài tay_áo k...
3,193,áo ba lỗ chất liệu cotton kho hàng gửi từ hà n...,áo ba lỗ chất_liệu cotton kho hàng gửi từ hà n...
4,1915,áo ba lỗ chiều dài tay áo không tay phong cách...,áo ba_lỗ chiều dài tay_áo không tay phong_cách...


In [24]:
# Tokenize(split) the sentences into words
products_gem = [[text for text in x.split()] for x in df_xl.product_name_wt]

In [25]:
len(products_gem)

10576

In [26]:
products_gem[:1]

[['áo',
  'ba_lỗ',
  'chiều',
  'dài',
  'tay_áo',
  'khác',
  'phong_cách',
  'thể_thao',
  'cơ_bản',
  'đường_phố',
  'nhiệt_đới',
  'tall',
  'fit',
  'có',
  'xuất_xứ',
  'việt',
  'nam',
  'dáng',
  'kiểu',
  'áo',
  'ôm',
  'sát',
  'chất_liệu',
  'cotton',
  'mẫu',
  'sọc',
  'tăm',
  'có',
  'gân',
  'nổi',
  'kho',
  'hàng',
  'gửi',
  'từ',
  'bà_rịa',
  'vũng_tàu',
  'best',
  'tank',
  'tops',
  'for',
  'men',
  'áo',
  'ba',
  'lỗ',
  'quốc_dân',
  'cho',
  'nam',
  'bảng',
  'bảng',
  'này',
  'là',
  'chuẩn',
  'mặc',
  'kiểu',
  'body',
  'nha',
  'quý_khách',
  'còn',
  'nếu',
  'khách',
  'nào',
  'không',
  'thích',
  'mặc',
  'ôm',
  'thì',
  'tăng',
  'lên',
  'nhé',
  'còn',
  'muốn',
  'mặc',
  'rộng',
  'rộng',
  'thì',
  'tăng',
  'lên',
  'bảng',
  'chỉ',
  'mang',
  'tính_chất',
  'tham_khảo',
  'vì',
  'có',
  'nhiều',
  'khách_thể_hình',
  'không',
  'được',
  'cân_đối',
  'để',
  'được',
  'tư_vấn',
  'chính_xác',
  'vui_lòng',
  'inbox',
  'cho',
  'shop

In [27]:
import re

In [28]:
products_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in products_gem] # số
products_gem_re = [[t.lower() for t in text if not t in ['', ' ', ',', '.', '...', '-',':', ';', '?', '%', '_%' , '(', ')', '+', '/', 'g', 'ml']] for text in  products_gem_re] # ký tự đặc biệt
products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re] # stopword

In [29]:
products_gem_re[:1]

[['áo',
  'ba_lỗ',
  'chiều',
  'tay_áo',
  'phong_cách',
  'thể_thao',
  'cơ_bản',
  'đường_phố',
  'nhiệt_đới',
  'tall',
  'fit',
  'xuất_xứ',
  'việt',
  'nam',
  'dáng',
  'kiểu',
  'áo',
  'ôm',
  'sát',
  'chất_liệu',
  'cotton',
  'mẫu',
  'sọc',
  'tăm',
  'gân',
  'nổi',
  'kho',
  'hàng',
  'gửi',
  'bà_rịa',
  'vũng_tàu',
  'best',
  'tank',
  'tops',
  'for',
  'men',
  'áo',
  'lỗ',
  'quốc_dân',
  'nam',
  'bảng',
  'bảng',
  'chuẩn',
  'mặc',
  'kiểu',
  'body',
  'nha',
  'quý_khách',
  'không',
  'thích',
  'mặc',
  'ôm',
  'mặc',
  'rộng',
  'rộng',
  'bảng',
  'tính_chất',
  'tham_khảo',
  'khách_thể_hình',
  'không',
  'cân_đối',
  'tư_vấn',
  'chính_xác',
  'vui_lòng',
  'inbox',
  'shop',
  'kích_thước',
  'vòng',
  'nách',
  'l',
  'xl',
  'dài_dài',
  'bảng',
  'mặc',
  'cơ_thể',
  'không',
  'ôm',
  'sát',
  'không',
  'rộng',
  'tùy',
  'sở_thích',
  'body',
  'chuẩn',
  'múi',
  'mặc',
  'ôm',
  'sát',
  'hạ',
  'mặc',
  'rộng_rãi',
  'thoải_mái',
  'bảng',


In [30]:
# Obtain the number of features based on dictionary: Use corpora.Dictionary
dictionary = corpora.Dictionary(products_gem_re)
dictionary.save('dictionary.dictionary')

In [31]:
# List of features in dictionary
dictionary.token2id

{'ba_lỗ': 0,
 'basic': 1,
 'best': 2,
 'body': 3,
 'bà_rịa': 4,
 'bèo_nhèo': 5,
 'bảng': 6,
 'bấm': 7,
 'cam_kết': 8,
 'chiều': 9,
 'chuẩn': 10,
 'chàng': 11,
 'chính_xác': 12,
 'chảy': 13,
 'chất': 14,
 'chất_liệu': 15,
 'co_giãn': 16,
 'cotton': 17,
 'cuốn_hút': 18,
 'cá_tính': 19,
 'cân_đối': 20,
 'cơ_bản': 21,
 'cơ_thể': 22,
 'cơ_địa': 23,
 'cứng': 24,
 'cực': 25,
 'cực_kỳ': 26,
 'dài_dài': 27,
 'dáng': 28,
 'fit': 29,
 'fom': 30,
 'for': 31,
 'form': 32,
 'giúp': 33,
 'gym': 34,
 'gân': 35,
 'gửi': 36,
 'hoàn': 37,
 'hàng': 38,
 'hình_ảnh': 39,
 'hút': 40,
 'hạ': 41,
 'hầu_như': 42,
 'inbox': 43,
 'kho': 44,
 'khoán': 45,
 'khách_thể_hình': 46,
 'không': 47,
 'khỏe': 48,
 'kiểu': 49,
 'kích_thước': 50,
 'l': 51,
 'lỗ': 52,
 'lỗi_thời': 53,
 'men': 54,
 'mix': 55,
 'mô_tả': 56,
 'múi': 57,
 'mẫu': 58,
 'mặc': 59,
 'nam': 60,
 'ngoại_lệ': 61,
 'nha': 62,
 'nhiệt_đới': 63,
 'nhão': 64,
 'nhấn': 65,
 'nhắn_tin': 66,
 'nách': 67,
 'năng_động': 68,
 'nổi': 69,
 'phong_cách': 70,
 'phái'

In [32]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)

In [33]:
feature_cnt

18069

In [34]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [35]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 5),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 2),
 (11, 2),
 (12, 3),
 (13, 1),
 (14, 2),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 2),
 (34, 1),
 (35, 3),
 (36, 1),
 (37, 1),
 (38, 3),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 11),
 (48, 1),
 (49, 2),
 (50, 1),
 (51, 1),
 (52, 2),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 2),
 (59, 9),
 (60, 3),
 (61, 1),
 (62, 2),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 2),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 3),
 (75, 1),
 (76, 3),
 (77, 1),
 (78, 3),
 (79, 3),
 (80, 2),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 2),
 (88, 1),
 (89, 1),
 (90, 2),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 2)

In [36]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus) 
tfidf.save('tfidf.tfidfmodel') 
# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus], 
                                            num_features = feature_cnt) 
index.save('index.docsim')

In [37]:
data_result = pd.DataFrame(index[tfidf[corpus]])

In [38]:
data_result

,0,1,2,3,4,5,6,7,8,9,...,10566,10567,10568,10569,10570,10571,10572,10573,10574,10575
0,1.000000,0.078570,0.111208,0.116535,0.162848,0.077709,0.155038,0.144310,0.068990,0.128068,...,0.046241,0.034251,0.022258,0.062858,0.042662,0.037111,0.015872,0.013002,0.033343,0.021720
1,0.078570,1.000000,0.073814,0.214967,0.049760,0.035172,0.098041,0.053707,0.037792,0.093210,...,0.022402,0.025703,0.064364,0.023202,0.018422,0.086308,0.096778,0.034943,0.063768,0.108164
2,0.111208,0.073814,1.000000,0.097637,0.239999,0.047624,0.130601,0.183527,0.067696,0.146558,...,0.038354,0.034038,0.037552,0.047251,0.032398,0.059707,0.035480,0.024093,0.047969,0.066117
3,0.116535,0.214967,0.097637,1.000000,0.074199,0.042917,0.161793,0.060822,0.049030,0.179951,...,0.047072,0.041206,0.077517,0.033551,0.017729,0.033994,0.017270,0.036444,0.028152,0.045879
4,0.162848,0.049760,0.239999,0.074199,1.000000,0.059789,0.108470,0.082566,0.127902,0.165887,...,0.019151,0.026517,0.071996,0.075155,0.050343,0.071783,0.098310,0.058896,0.060694,0.050289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10571,0.037111,0.086308,0.059707,0.033994,0.071783,0.034054,0.058912,0.044500,0.049422,0.036108,...,0.158883,0.337569,0.244889,0.084189,0.096295,1.000000,0.249557,0.169247,0.279238,0.266621
10572,0.015872,0.096778,0.035480,0.017270,0.098310,0.030436,0.027501,0.039260,0.066002,0.044885,...,0.224561,0.189038,0.193503,0.099977,0.074559,0.249557,1.000001,0.136068,0.266865,0.298915
10573,0.013002,0.034943,0.024093,0.036444,0.058896,0.016897,0.021170,0.020491,0.035424,0.041717,...,0.094761,0.128617,0.107240,0.040664,0.044490,0.169247,0.136068,1.000000,0.125857,0.082851
10574,0.033343,0.063768,0.047969,0.028152,0.060694,0.036337,0.046471,0.038652,0.061701,0.043703,...,0.202359,0.251539,0.223374,0.092876,0.103225,0.279238,0.266865,0.125857,1.000000,0.292084


# TRƯỜNG HỢP 1:
# Khi người dùng chọn 1 sản phẩm, đề xuất các sản phẩm tương tự
# Giả sử, người dùng chọn 1 sản phẩm có index = 0

In [39]:
data_result[0].sort_values(ascending = False).head(4)

0       1.000000
5490    0.233629
362     0.213355
193     0.204788
Name: 0, dtype: float32

In [40]:
# Vậy sản phẩm liên quan nhất là sản phẩm có index = 926
df.iloc[926]

product_id                                                    211576
description        áo khoác áo khoác mùa đông áo choàng chất liệu...
product_name_wt    áo_khoác áo_khoác mùa đông áo_choàng chất_liệu...
Name: 926, dtype: object

In [41]:
df_full = pd.read_csv('data_full_12_4_2023.csv')

In [42]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10604 entries, 0 to 10603
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product_id       10604 non-null  int64  
 1   description      10604 non-null  object 
 2   product_name_wt  10604 non-null  object 
 3   product_name     10604 non-null  object 
 4   sub_category     10604 non-null  object 
 5   link             10604 non-null  object 
 6   image            10604 non-null  object 
 7   price            10604 non-null  float64
 8   rating           10604 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 745.7+ KB


In [43]:
# Hàm xuất thông tin sản phẩm
def san_pham_de_xuat(id,data):
    print('product_id:',data.iloc[id,0])
    #print('description:',data.iloc[id,1])
    print('product_name:',data.iloc[id,3])
    print('price:',data.iloc[id,8])
    print('rating:',data.iloc[id,8])

In [44]:
san_pham_de_xuat(926,df_full)

product_id: 211576
product_name: Áo Chống Nắng Kẻ Nam 1 Lớp
price: 5.0
rating: 5.0


In [45]:
df_full.iloc[926,7]

56000.0

In [46]:
lst_sub = df_full['sub_category'].unique().tolist()
df_full['sub_category_id'] = df_full['sub_category'].map(lambda x:lst_sub.index(x)+1)
df_full['item_id'] = df_full.apply(lambda x: int(str(x['product_id'])+str(x['sub_category_id']).zfill(2)),axis=1)
df_full.head()

,product_id,description,product_name_wt,product_name,sub_category,link,image,price,rating,sub_category_id,item_id
0,190,áo ba lỗ chiều dài tay áo khác phong cách thể ...,áo ba_lỗ chiều dài tay_áo khác phong_cách thể_...,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,https://shopee.vn/%C3%81o-ba-l%E1%BB%97-thun-g...,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdf...,86250.0,4.9,1,19001
1,191,áo ba lỗ xuất xứ việt nam tên tổ chức chịu trá...,áo ba lỗ xuất_xứ việt nam tên tổ_chức chịu trá...,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Tr...,https://cf.shopee.vn/file/c7ea4c6574dc79be6b26...,26800.0,4.9,1,19101
2,192,áo ba lỗ thương hiệu tyasuo chiều dài tay áo k...,áo ba lỗ thương_hiệu tyasuo chiều dài tay_áo k...,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Ty...,https://cf.shopee.vn/file/6f93bcda10efe374f8cc...,39500.0,4.8,1,19201
3,193,áo ba lỗ chất liệu cotton kho hàng gửi từ hà n...,áo ba lỗ chất_liệu cotton kho hàng gửi từ hà n...,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,https://shopee.vn/%C3%81O-BA-L%E1%BB%96-H%C3%8...,https://cf.shopee.vn/file/1d7ed5e34bff8bc8b49a...,16500.0,4.8,1,19301
4,1915,áo ba lỗ chiều dài tay áo không tay phong cách...,áo ba_lỗ chiều dài tay_áo không tay phong_cách...,40-105kg áo 3 lỗ/tanktop/sát nách nam nữ thời ...,Áo Ba Lỗ,https://shopee.vn/40-105kg-%C3%A1o-3-l%E1%BB%9...,https://cf.shopee.vn/file/0f84cc93f49aa652dd28...,59000.0,4.8,1,191501


In [47]:
len(df_full)

10604

In [48]:
df_full.to_csv('data_full_info_12_4_2023.csv',index = False)

In [49]:
#df_full = pd.read_csv('data_full_info.csv')
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10604 entries, 0 to 10603
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product_id       10604 non-null  int64  
 1   description      10604 non-null  object 
 2   product_name_wt  10604 non-null  object 
 3   product_name     10604 non-null  object 
 4   sub_category     10604 non-null  object 
 5   link             10604 non-null  object 
 6   image            10604 non-null  object 
 7   price            10604 non-null  float64
 8   rating           10604 non-null  float64
 9   sub_category_id  10604 non-null  int64  
 10  item_id          10604 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 911.4+ KB


In [50]:
df_full.loc[df_full['item_id'] == 19401]

,product_id,description,product_name_wt,product_name,sub_category,link,image,price,rating,sub_category_id,item_id


In [51]:
df_top5 = (data_result[0].sort_values(ascending=False))[:6]

In [52]:
df_top5

0       1.000000
5490    0.233629
362     0.213355
193     0.204788
300     0.199331
164     0.192154
Name: 0, dtype: float32

In [53]:
df_top5 = df_top5.reset_index()

In [54]:
df_top5['index']

0       0
1    5490
2     362
3     193
4     300
5     164
Name: index, dtype: int64

In [55]:
# Hàm hiển thị 5 sản phẩm đề xuất
def top_5_san_pham_de_xuat(index):
    print('Đề xuất các sản phẩm tương tự: ')
    print('* '*40)
    for i in index:
        san_pham_de_xuat(i,df_full)
        print('* '*40)

In [56]:
top_5_san_pham_de_xuat(df_top5['index'])

Đề xuất các sản phẩm tương tự: 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 190
product_name: Áo ba lỗ thun gân ,form body tôn dáng
price: 4.9
rating: 4.9
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 118
product_name: Quần Jean nam Đen Xả Lai Rách Gối Unisex - quần bò nam hàn quốc
price: 4.8
rating: 4.8
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 19918
product_name: Áo Ba Lỗ Nam TankTop Adidas Sọc Eo 5 Màu Vải Thể Thao Siêu Mát Hàng Xuất Cực Xịn
price: 5.0
rating: 5.0
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 19270
product_name: [Xả Kho, Bán Lỗ] Áo ba lỗ sát nách nam cổ tim nhiều màu chất cotton thun lạnh 4 chiều mềm mịn dáng body thời trang
price: 4.5
rating: 4.5
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 19692
product_name: [ ÁO TANKTOP ] 

# TRƯỜNG HỢP 2:
# Khi người dùng gõ một nội dung tìm kiếm có mô tả sơ lược về một sản phẩm nào đó

In [57]:
#tim = input("Nhập nội dung tìm: ")
tim = 'kính mát'

In [58]:
def remove_long_words(text):
    words = text.split()
    for word in words:
        if len(word) > 7:
            words.remove(word)
    return ' '.join(words)

In [59]:
def clean_text(text):
    text_clean = str(text).lower()
    #Loại bỏ thông tin liên quan đến chi tiết như Xuất xứ, danh mục, kho hàng
    if 'danh mục\n' in text_clean: 
        text_clean = text_clean[text_clean.index("danh mục\n"):]
    elif 'thông tin sản phẩm\n' in text_clean: 
        text_clean = text_clean[text_clean.index("thông tin sản phẩm\n"):]
    elif "mô tả sản phẩm\n" in text_clean: 
        text_clean = text_clean[text_clean.index("mô tả sản phẩm\n"):]
    elif 'danh mục\n' in text_clean:
        text_clean = text_clean[text_clean.index("danh mục\n"):]
    elif 'shopee\n' in text_clean:
        text_clean = text_clean[text_clean.index("shopee\n"):]
    elif 'xuất xứ\n' in text_clean:
        text_clean = text_clean[text_clean.index("xuất xứ\n"):]
    elif 'kho hàng\n' in text_clean:
        text_clean = text_clean[text_clean.index("kho hàng\n"):]
    elif 'thời trang nam\n' in text_clean:
        text_clean = text_clean[text_clean.index("thời trang nam\n"):]
    elif "\n\n" in text_clean: 
        text_clean = text_clean[text_clean.index("\n\n") + 4:]
    elif "\ngửi từ\n" in text_clean: 
        text_clean = text_clean[text_clean.index("\ngửi từ\n") + len("\ngửi từ\n"):]
    elif "\ngửi từ\n" in text_clean: 
        text_clean = text_clean[text_clean.index("\ngửi từ\n") + len("\ngửi từ\n"):]
    #Loại bỏ phần size
    text_clean = re.sub(r"\nsize[^\n]*","",text_clean)

    #Loại bỏ các hastag
    text_clean = re.sub(r"#[^#]*","", text_clean)
    #Loại bỏ các ký tự không hợp lệ
    text_clean = re.sub(r"\n", " ", text_clean) 
    text_clean = emoji.replace_emoji(text_clean)
    text_clean = re.sub('[\.\:\,\-\-\-\+\d\!\%\...\.\"\*\>\<\^\&\/\[\]\(\)\=\~]',' ', text_clean)
    #Loại bỏ các từ không cần thiết
    text_clean = re.sub('\ss\s|\sm\s|\sl\s|\sxl|xxl|xxxl|xxxxl|2x1|3x1|4xl|size|\smm\s|\scm\s|\sm\s|\sg\s|\skg\s',' ', text_clean)
    #Loại bỏ khoảng trắng thừa
    text_clean = re.sub('\s+',' ', text_clean)
    
    text_clean = re.sub('[\s]{2,}', ' ', text_clean)
    text_clean = re.sub('[\s]{3,}', ' ', text_clean)
    text_clean = re.sub('^[\s]{1,}', '', text_clean)
    text_clean = re.sub('[\s]{1,}$', '', text_clean)
    text_clean = re.sub('[^\w\s]', '', text_clean)    
    text_clean = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '',text_clean)
    text_clean = remove_long_words(text_clean)
    text_clean = re.sub('danh mục shopee thời trang nam', ' ', text_clean)
    return text_clean

In [60]:
# Xử lý nội dung người dùng nhập
tim = clean_text(tim)

In [61]:
tim

'kính mát'

In [62]:
tim = pd.Series(tim)

In [63]:
tim = pd.DataFrame(tim,columns=['user_description'])
tim

,user_description
0,kính mát


In [64]:
# word_tokenize
tim["user_description_wt"] = tim["user_description"].apply(lambda x: word_tokenize(x, format="text"))
tim

,user_description,user_description_wt
0,kính mát,kính_mát


In [65]:
# sản phẩm đang xem
name_description_pre = tim['user_description_wt'].to_string(index=False)
name_description_pre

'kính_mát'

In [66]:
# Tokenize(split) the sentences into words
product_gem = [[text for text in x.split()] for x in tim.user_description_wt]

In [67]:
len(product_gem)
print(product_gem)

[['kính_mát']]


In [68]:
# remove some special elements in texts
products_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in products_gem] # số
products_gem_re = [[t.lower() for t in text if not t in ['', ' ', ',', '.', '...', '-',':', ';', '?', '%', '_%' , '(', ')', '+', '/', 'g', 'ml']] for text in  products_gem_re] # ký tự đặc biệt

In [69]:
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [70]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus) 

# tính toán sự tương tự trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus], 
                                            num_features = feature_cnt) 


In [71]:
view_product = name_description_pre.split()
view_product

['kính_mát']

In [72]:
# Convert search words into Sparse Vectors
kw_vector = dictionary.doc2bow(view_product)

In [73]:
kw_vector

[(7397, 1)]

In [74]:
# similarity calculation and select top 5
sim = index[tfidf[kw_vector]]
sim

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [75]:
results = pd.DataFrame(sim,columns=['score'])
results

,score
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
10571,0.0
10572,0.0
10573,0.0
10574,0.0


In [76]:
#df_top5 = (results['score'].sort_values(ascending=False))
df_top5 = (results['score'].sort_values(ascending=False))[:6]

In [77]:
df_top5

6765    0.583100
6996    0.527539
7438    0.527539
7012    0.515673
6627    0.515673
6822    0.514436
Name: score, dtype: float32

In [78]:
df_top5 = df_top5.reset_index()
df_top5['index']

0    6765
1    6996
2    7438
3    7012
4    6627
5    6822
Name: index, dtype: int64

In [79]:
top_5_san_pham_de_xuat(df_top5['index'])

Đề xuất các sản phẩm tương tự: 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 12294
product_name: Kính mát nam Tisselly M05 lái xe thời trang gọng và tròng nhựa dẻo cao cấp
price: 4.7
rating: 4.7
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 12549
product_name: KMUYTIN Kính Mắt Thời Trang Nam Au 01 Mắt Hà Hơi
price: 5.0
rating: 5.0
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 121046
product_name: Kính nam sang chảnh 3 sọc
price: 5.0
rating: 5.0
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 12566
product_name: [siêu rẻ] Kính nam tròng phin mes đi ngày và đêm không viền
price: 5.0
rating: 5.0
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
product_id: 12136
product_name: Kính Râm Nam Vuông, Mắt Kính Phân Cực Chống Chói, Đi Xe Ban Ngày Gọng Nhựa Siêu Nhẹ AK022 -AORON
pri